# Phase 1 Definition of Done (DoD)

**Project**: Causal Uplift Marketing Analysis (Hillstrom Dataset)  
**Phase**: Phase 1 — Data Pipeline & Feature Engineering  
**Status**: ✅ ALL CRITERIA PASSED

---

## Overview

本文档记录 Phase 1 的验收结果。Phase 1 的目标是搭建数据管道，从原始 Hillstrom 数据集出发，完成数据清洗和特征工程，为后续的因果推断建模（PSM 和 Uplift Modeling）准备好输入数据。

验收标准共 5 项，均已通过：

| # | Criterion | Status |
|---|-----------|--------|
| 1 | Data Pipeline Validation | ✅ PASSED |
| 2 | Data Flow Integration | ✅ PASSED |
| 3 | Notebook Reproducibility | ✅ PASSED |
| 4 | Artifact Completeness | ✅ PASSED |
| 5 | Documentation Completeness | ✅ PASSED |

---

## Criterion 2: Data Flow Integration 

**核心验证**：调用 `build_features()`，确认从清洗数据到特征矩阵的完整数据流可以跑通。

需要验证的几个关键点：
- 输入输出行数一致（64,000 行）
- 所有特征列都是数值类型，没有 NaN 或 inf
- `treatment`、`conversion`、`spend` 这三列必须保留
- `segment`、`channel`、`zip_code`、`visit` 这几列必须被删掉（避免数据泄露或引入混淆变量）

> 注：`visit` 是 treatment 的下游中介变量（mediator），如果保留会低估 ATE，所以必须在特征工程阶段删除。

### 时序污染检测（Temporal Contamination Check）

在 `load_and_clean()` 中加入了断言检查，验证 `visit` 是否为真正的 Post-treatment Variable：
- Control 组 visit rate: 10.62%
- Treatment 组 visit rate: 16.70%
- 符合预期（treatment 组略高），说明不存在 Pre-treatment Visit

### 敏感性分析（Sensitivity Analysis）

如果在真实业务场景中遇到时序污染（control 组 visit rate 异常高），处理策略：

1. **重新定义 Mediator**：将 `visit` 拆分为 `visit_pre`（Confounder，必须控制）和 `visit_post`（Mediator，必须排除）
2. **使用 DID 估计量**：通过 Difference-in-Differences 消除 Pre-treatment Visit 带来的选择偏差
3. **退回 PSM**：如果无法拆分时序，将 `visit` 作为 Confounder 控制，估计 ATT 而非 ATE

In [1]:
from pathlib import Path
import sys
import os
import pandas as pd
import yaml

# project root detection (works whether run from notebooks/ or project root)
project_root = Path.cwd()
if project_root.name == 'notebooks':
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.data_utils import build_features

os.chdir(project_root)

with open("configs/config.yml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# load cleaned data
df = pd.read_csv(config["paths"]["cleaned_data"])
print(f"cleaned data loaded: {df.shape}")

# run build_features
features_df = build_features(df, config)
print(f"features generated: {features_df.shape}")
print(f"columns: {list(features_df.columns)}")

# basic checks
assert len(df) == len(features_df), "row count mismatch!"
assert features_df.isnull().sum().sum() == 0, "NaN values found!"
assert "treatment" in features_df.columns
assert "visit" not in features_df.columns, "mediator column should be dropped"

print("\n✅ Criterion 2 PASSED")

cleaned data loaded: (64000, 13)
features generated: (64000, 16)
columns: ['recency', 'history', 'mens', 'womens', 'newbie', 'conversion', 'spend', 'treatment', 'channel_Multichannel', 'channel_Phone', 'channel_Web', 'zip_Rural', 'zip_Surburban', 'zip_Urban', 'history_log', 'is_both_gender']

✅ Criterion 2 PASSED


---

## Criterion 3: Notebook Reproducibility 

所有 notebook 从头到尾重新执行，没有报错。

路径处理用的是 `Path.cwd()` 动态检测，不依赖硬编码路径，换机器也能跑。依赖项都在 `requirements.txt` 里。

已测试的 notebooks：
- `01_data_ingestion_and_eda.ipynb`
- `02_bias_exposure_and_naive_ate.ipynb`
- `Phase1_DoD.ipynb`（本文档）

---

## Criterion 4: Artifact Completeness 

Phase 1 需要产出的文件都已生成：

| File | Path | 说明 |
|------|------|------|
| Raw snapshot | `data/raw/hillstrom_raw_text_*.csv` | 原始数据的时间戳快照，保证数据血缘可追溯 |
| Cleaned data | `data/processed/hillstrom_cleaned.csv` | 经过 DQ 检查的清洗数据 |
| Feature matrix | `data/processed/hillstrom_features.csv` | 特征工程输出，16 列，全数值型 |
| EDA figures | `outputs/figures/eda_*.png` | EDA 可视化图表 |

---

## Criterion 5: Documentation Completeness 

代码、配置、文档都有对应的说明：

- `src/data_utils.py` — 两个核心函数都有 docstring 和类型注解
- `configs/config.yml` — 所有参数都有注释说明
- `docs/Phase1_Execution_PRD.md` — Phase 1 的需求文档
- 本文档 — Phase 1 验收记录

---

## Summary

In [2]:
print("Phase 1 DoD — All 5 criteria passed")
print()
print("Deliverables:")
print("  - data pipeline: load_and_clean() + build_features()")
print("  - 64,000 rows, 16 features, no NaN/inf")
print("  - raw data snapshots for reproducibility")
print("  - EDA with covariate balance check (SMD)")
print()
print("Next: Phase 2 — Propensity Score Matching & Uplift Modeling")

Phase 1 DoD — All 5 criteria passed

Deliverables:
  - data pipeline: load_and_clean() + build_features()
  - 64,000 rows, 16 features, no NaN/inf
  - raw data snapshots for reproducibility
  - EDA with covariate balance check (SMD)

Next: Phase 2 — Propensity Score Matching & Uplift Modeling
